In [ ]:
import os
import copy

import numpy as np
import gsw   as gsw

from netCDF4 import Dataset

In [ ]:
# Diretories where data is stored:

datadir = '/gws/nopw/j04/nemo_vol1/jmecking/en4/v4.2.2_c14/' # Directory with original downloaded EN4 data
savedir = '/gws/nopw/j04/nemo_vol1/jmecking/en4/v4.2.2_c14/density/'  # Directory to store density data

In [ ]:
# Compute potential and in-situ density:
yr = np.arange(1900,2025)
ny = np.size(yr,axis=0)

for yy in range(0,ny):
    for mm in range(0,12):
        infile  = (datadir + 'EN.4.2.2.f.analysis.c14.' + str(yr[yy]) + str(mm+1).zfill(2) + '.nc')
        outfile = (savedir + 'EN.4.2.2.f.analysis.c14.' + str(yr[yy]) + str(mm+1).zfill(2) + '_density.nc')
        if os.path.isfile(infile):
            if not os.path.isfile(outfile):
                print(infile)
                # Read in temperature and salinity:
                ncid = Dataset(infile,'r')
                pt  = np.squeeze(ncid.variables['temperature'][:,:,:,:])
                SP  = np.squeeze(ncid.variables['salinity'][:,:,:,:])
                lat = ncid.variables['lat'][:]
                lon = ncid.variables['lon'][:]
                depth = ncid.variables['depth'][:]
                time  = ncid.variables['time'][:]
                time_bnds = ncid.variables[ncid.variables['time'].bounds][:,:]
                ncid.close()

                # Set required variables:
                LON,LAT = np.meshgrid(lon,lat)
                LON = np.tile(LON,(np.size(depth),1,1))
                LAT = np.tile(LAT,(np.size(depth),1,1))
                DEP = np.swapaxes(np.tile(depth,(np.size(lon),np.size(lat),1)),0,2)
                p   = gsw.p_from_z(-DEP,LAT)
                
                # Compute densities:
                SA     = gsw.SA_from_SP(SP, p, LON, LAT)
                CT     = gsw.conversions.CT_from_pt(SA, pt-273.15)
                sigma0 = gsw.density.sigma0(SA, CT)
                rho    = gsw.density.rho(SA, CT, p)

                # Write to file:
                ncid = Dataset(outfile,'w')
                
                # coordinates:
                ncid.createDimension('lat'  ,len(lat))
                ncid.createDimension('lon'  ,len(lon))
                ncid.createDimension('depth',len(depth))
                ncid.createDimension('bnds' ,2)
                ncid.createDimension('time' ,None)

                # variables:
                ncid.createVariable('lat'      ,'f8' ,('lat',))
                ncid.createVariable('lon'      ,'f8' ,('lon',))
                ncid.createVariable('depth'    ,'f8' ,('depth',))
                ncid.createVariable('time'     ,'f8' ,('time',))
                ncid.createVariable('time_bnds','f8' ,('time','bnds',))
                ncid.createVariable('rho'      ,'f8' ,('time','depth','lat','lon',))
                ncid.createVariable('sigma0'   ,'f8' ,('time','depth','lat','lon',))

                # fill variables:
                ncid.variables['lat'][:]          = lat
                ncid.variables['lon'][:]          = lon
                ncid.variables['depth'][:]        = depth
                ncid.variables['time'][:]         = time
                ncid.variables['time_bnds'][:]    = time_bnds
                ncid.variables['rho'][0,:,:,:]    = rho
                ncid.variables['sigma0'][0,:,:,:] = 1000 + sigma0

                # Close netcdf file:
                ncid.close()